In [1]:
import h5py
import os,os.path as osp
import numpy as np

C:\Users\sarak\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D, GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [3]:
data_file = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

f = h5py.File(data_file,'r')
for key in f.keys():
    print(key)

block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [4]:
def custom_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)
    
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)
    
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)
    
   
    x = Flatten(name='flatten')(x)
    x = Dense(64, activation='relu', name='fc1')(x)
    x = Dropout(0.5, name='dropout1')(x)
    x = Dense(32, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(4, activation='relu', name='fc3')(x)
    
    model = Model(inputs=input_img, outputs=x)
    return model

In [5]:
model = custom_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ImageInput (InputLayer)      (None, 224, 224, 3)       0         
_________________________________________________________________
Conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
Conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
Conv2_1 (SeparableConv2D)    (None, 112, 112, 128)     8896      
_________________________________________________________________
Conv2_2 (SeparableConv2D)    (None, 112, 112, 128)     17664     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
__________

In [6]:
# Select the layers for which you want to set weight.
# weight, block

w,b = f['block1_conv1']['block1_conv1_W_1:0'], f['block1_conv1']['block1_conv1_b_1:0']
model.layers[1].set_weights = [w,b]

w,b = f['block1_conv2']['block1_conv2_W_1:0'], f['block1_conv2']['block1_conv2_b_1:0']
model.layers[2].set_weights = [w,b]

w,b = f['block2_conv1']['block2_conv1_W_1:0'], f['block2_conv1']['block2_conv1_b_1:0']
model.layers[4].set_weights = [w,b]

w,b = f['block2_conv2']['block2_conv2_W_1:0'], f['block2_conv2']['block2_conv2_b_1:0']
model.layers[5].set_weights = [w,b]

f.close()
# model.summary()

In [7]:
for i,layer in enumerate(model.layers):
    print(i,layer.name)

0 ImageInput
1 Conv1_1
2 Conv1_2
3 pool1
4 Conv2_1
5 Conv2_2
6 pool2
7 Conv3_1
8 bn1
9 Conv3_2
10 bn2
11 Conv3_3
12 pool3
13 flatten
14 fc1
15 dropout1
16 fc2
17 dropout2
18 fc3


In [8]:
def print_structure(weight_file_path):
    """
    Prints out the structure of HDF5 file.

    Args:
      weight_file_path (str) : Path to the file to analyze
    """
    f = h5py.File(weight_file_path)
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))

        if len(f.items())==0:
            return 

        for layer, g in f.items():
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items():
                print("      {}: {}".format(key, value))

            print("    Dataset:")
            for p_name in g.keys():
                param = g[p_name]
                print("      {}: {}".format(p_name, param.shape))
    finally:
        f.close()
        

weight_file_path= 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
print_structure(weight_file_path)

vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5 contains: 
Root attributes:
  layer_names: [b'block1_conv1' b'block1_conv2' b'block1_pool' b'block2_conv1'
 b'block2_conv2' b'block2_pool' b'block3_conv1' b'block3_conv2'
 b'block3_conv3' b'block3_pool' b'block4_conv1' b'block4_conv2'
 b'block4_conv3' b'block4_pool' b'block5_conv1' b'block5_conv2'
 b'block5_conv3' b'block5_pool']
  block1_conv1
    Attributes:
      weight_names: [b'block1_conv1_W_1:0' b'block1_conv1_b_1:0']
    Dataset:
      block1_conv1_W_1:0: (3, 3, 3, 64)
      block1_conv1_b_1:0: (64,)
  block1_conv2
    Attributes:
      weight_names: [b'block1_conv2_W_1:0' b'block1_conv2_b_1:0']
    Dataset:
      block1_conv2_W_1:0: (3, 3, 64, 64)
      block1_conv2_b_1:0: (64,)
  block1_pool
    Attributes:
      weight_names: []
    Dataset:
  block2_conv1
    Attributes:
      weight_names: [b'block2_conv1_W_1:0' b'block2_conv1_b_1:0']
    Dataset:
      block2_conv1_W_1:0: (3, 3, 64, 128)
      block2_conv1_b_1:0: (128,)
  

In [11]:
data_file = 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

f = h5py.File(data_file,'r')
for key in f.keys():
    print(key)

block1_conv1
block1_conv2
block1_pool
block2_conv1
block2_conv2
block2_pool
block3_conv1
block3_conv2
block3_conv3
block3_pool
block4_conv1
block4_conv2
block4_conv3
block4_pool
block5_conv1
block5_conv2
block5_conv3
block5_pool


In [12]:
a = list(f.keys())[0]
a

'block1_conv1'

In [13]:
list(f['block1_conv1'])

['block1_conv1_W_1:0', 'block1_conv1_b_1:0']

In [16]:
list(f['block1_conv1']['block1_conv1_W_1:0'])

[array([[[ 0.42947057,  0.11727387,  0.03401296,  0.35422093,
          -0.0865837 ,  0.22968295,  0.06683242,  0.04133838,
           0.13026784,  0.03291035,  0.09615457, -0.09222532,
          -0.15587331,  0.0522968 , -0.08324417,  0.15728769,
           0.0291196 ,  0.05094835,  0.19055748, -0.04372229,
           0.3618031 ,  0.23902874,  0.16667192, -0.18046746,
           0.04404821,  0.09592837, -0.04381131, -0.1884835 ,
          -0.04399502, -0.18985446, -0.23386982,  0.12836744,
          -0.11059975,  0.00571336, -0.24479656, -0.30530283,
           0.060938  , -0.32816252,  0.1711953 ,  0.10265407,
          -0.11063665,  0.33012986, -0.05869728,  0.18478717,
           0.01050853,  0.10668027, -0.13086924, -0.36283946,
           0.06440807,  0.09335972,  0.00401619,  0.37814805,
           0.05268804, -0.27078936,  0.11060741, -0.11819526,
           0.04298935,  0.35323623, -0.28468162, -0.20289008,
          -0.284944  , -0.13224158, -0.05334752,  0.00757738],
       

In [17]:
for name in f['block1_conv1']:
    print(name)

block1_conv1_W_1:0
block1_conv1_b_1:0


In [18]:
f['block1_conv1']['block1_conv1_W_1:0']

<HDF5 dataset "block1_conv1_W_1:0": shape (3, 3, 3, 64), type "<f4">

In [19]:
f['block1_conv1']['block1_conv1_W_1:0'].shape

(3, 3, 3, 64)

In [20]:
# # Get weights list
for layer in model.layers:
    weights = layer.get_weights()
    print(weights)

[]
[array([[[[ 0.08822162, -0.05478676, -0.04767516, ..., -0.05844746,
          -0.08675529,  0.09208049],
         [-0.09771135,  0.07980816,  0.06711569, ...,  0.06988895,
          -0.00913911,  0.08065508],
         [ 0.06368691,  0.08251965, -0.0448093 , ...,  0.0031678 ,
          -0.01821324,  0.05796196]],

        [[ 0.00287884,  0.09241934,  0.09942937, ...,  0.08606778,
           0.04941021, -0.08302993],
         [-0.01084065,  0.02448727, -0.00485793, ...,  0.09462199,
           0.0630112 , -0.06030244],
         [ 0.09773697, -0.03146888, -0.08065537, ..., -0.08587672,
          -0.01257446,  0.05253983]],

        [[-0.05201442,  0.07177114,  0.09942417, ...,  0.0562418 ,
           0.00953404, -0.06677477],
         [-0.00608002,  0.09766081, -0.07189909, ...,  0.00744441,
          -0.02028445, -0.06325506],
         [-0.065963  ,  0.0893316 ,  0.03015414, ..., -0.0016856 ,
           0.06126915,  0.06619248]]],


       [[[ 0.09710437, -0.05576777, -0.083464  , ...

[array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1